In [1]:
from pyspark.sql import SparkSession
from delta import *

In [2]:
builder = SparkSession.builder.appName("MiCuadernoDeConsultas") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

In [3]:
# configure_spark_with_delta_pip ya que esto es en un entorno venv
spark = configure_spark_with_delta_pip(builder).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/17 13:51:59 WARN Utils: Your hostname, Zidnz, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/17 13:51:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/zidnz/proyecto_fraude/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/zidnz/.ivy2.5.2/cache
The jars for the packages stored in: /home/zidnz/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-acc11e87-5e4c-42ac-b4cd-b92576748866;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 238ms :: artifacts dl 10ms
	:: modules in use:
	io.

In [4]:
ruta_tabla_delta = "/home/zidnz/DanaPP/proyecto_fraude/deltaLake"

In [5]:
print(f"Cargando tabla desde: {ruta_tabla_delta}")
df = spark.read.format("delta").load(ruta_tabla_delta)

Cargando tabla desde: /home/zidnz/DanaPP/proyecto_fraude/deltaLake


In [6]:
df.createOrReplaceTempView("transacciones")

In [7]:
spark.sql("SELECT * FROM transacciones LIMIT 5").show()

25/11/17 13:52:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+----------+--------------+-------------+----------+-------+------+----------------+----------------+--------------+--------------------+-------------------+--------------------+------------+---------------+-----------------+---------------+----------+---------------+
|id_transaccion|id_cliente|nombre_cliente|cuenta_origen|     fecha|  monto|divisa|tipo_transaccion|          ciudad|cuenta_destino|           categoria|    hora_movimiento| concepto_movimiento|saldo_previo|saldo_posterior|canal_transaccion|     medio_pago|   estatus|fraude_probable|
+--------------+----------+--------------+-------------+----------+-------+------+----------------+----------------+--------------+--------------------+-------------------+--------------------+------------+---------------+-----------------+---------------+----------+---------------+
|         27961|     18780|   Luis Garcia|      1.38E17|20/04/2025| 237.44|   MXN| Compra en linea|Ciudad de Mexico|       1.51E16|             Cons

In [8]:
# consulta 1: Cantidad de transacciones dependiendo de la variable fraude_probable
spark.sql("""
    SELECT 
        fraude_probable, 
        COUNT(*) as total_transacciones,
        AVG(monto) as monto_promedio
    FROM transacciones
    GROUP BY fraude_probable
""").show()

+---------------+-------------------+------------------+
|fraude_probable|total_transacciones|    monto_promedio|
+---------------+-------------------+------------------+
|             No|             194921|  936.307703992898|
|             Si|               5079|44318.581549517636|
+---------------+-------------------+------------------+



In [9]:
# Consulta 2: Top 15 transacciones MÁS ALTAS
spark.sql("""
    SELECT id_cliente, monto, divisa, fecha, ciudad, categoria
    FROM transacciones 
    ORDER BY monto DESC 
    LIMIT 15
""").show()

+----------+--------+------+----------+----------------+--------------------+
|id_cliente|   monto|divisa|     fecha|          ciudad|           categoria|
+----------+--------+------+----------+----------------+--------------------+
|      1200|99998.74|   MXN|21/09/2024|         Tijuana|             Ingreso|
|      4844|99998.74|   MXN|18/05/2025|       Monterrey|              Ahorro|
|     21678| 99998.7|   MXN|12/10/2024|Ciudad de Mexico|             Remesas|
|     27315|99997.53|   MXN|03/06/2024|       Monterrey|             Ingreso|
|     18424|99995.21|   MXN|04/02/2024|          Puebla|             Remesas|
|     10279|99994.87|   MXN|28/10/2024|Ciudad de Mexico|              Ahorro|
|     26539|99993.71|   MXN|31/12/2024|     Guadalajara|              Ahorro|
|     10099| 99991.2|   MXN|07/12/2024|Ciudad de Mexico|             Ingreso|
|     22933|99989.13|   MXN|03/08/2024|Ciudad de Mexico|             Remesas|
|     17971|99988.44|   MXN|13/09/2025|         Tijuana|        

In [10]:
# Consulta 4: Top 10 clientes con MÁS transacciones
spark.sql("""
    SELECT id_cliente, COUNT(*) as numero_de_transacciones
    FROM transacciones 
    GROUP BY id_cliente 
    ORDER BY numero_de_transacciones DESC 
    LIMIT 10
""").show()

+----------+-----------------------+
|id_cliente|numero_de_transacciones|
+----------+-----------------------+
|     17140|                     20|
|     14648|                     19|
|     12884|                     19|
|     18718|                     19|
|      2602|                     18|
|     11467|                     18|
|     26725|                     18|
|     16044|                     18|
|      6800|                     18|
|     16799|                     17|
+----------+-----------------------+



In [11]:
# Consulta 5: Transacciones fuera de hora (de madrugada)
spark.sql("""
    SELECT id_cliente, monto, hora_movimiento, canal_transaccion
    FROM transacciones
    WHERE hour(hora_movimiento) >= 2 AND hour(hora_movimiento) <= 5 
    ORDER BY monto DESC
    LIMIT 10
""").show()

+----------+--------+-------------------+-----------------+
|id_cliente|   monto|    hora_movimiento|canal_transaccion|
+----------+--------+-------------------+-----------------+
|     10279|99994.87|2025-11-17 03:21:22|Sucursal bancaria|
|     18748|99976.38|2025-11-17 02:43:17|Sucursal bancaria|
|     17202|99963.87|2025-11-17 03:08:41|Sucursal bancaria|
|      9588|99962.34|2025-11-17 05:19:27|              Web|
|     20675|99960.81|2025-11-17 05:27:54|              Web|
|     18334|99959.86|2025-11-17 02:13:14|     Corresponsal|
|     23189|99947.85|2025-11-17 02:37:22|Sucursal bancaria|
|      4967| 99924.7|2025-11-17 04:16:55|              Web|
|     27766|99921.06|2025-11-17 04:12:55|     Corresponsal|
|     14763|99804.81|2025-11-17 04:53:33|Sucursal bancaria|
+----------+--------+-------------------+-----------------+



In [12]:
# Consulta 7: Canales de transacción inusuales
spark.sql("""
    SELECT 
        canal_transaccion, 
        COUNT(*) as total, 
        AVG(monto) as monto_promedio,
        MAX(monto) as monto_maximo
    FROM transacciones
    GROUP BY canal_transaccion
    ORDER BY monto_promedio DESC
""").show()

+-----------------+-----+------------------+------------+
|canal_transaccion|total|    monto_promedio|monto_maximo|
+-----------------+-----+------------------+------------+
|Sucursal bancaria|23223| 5863.492222365755|     99998.7|
|     Corresponsal| 3877|   4849.4779778179|    99998.74|
|              Web|60195|1632.8690291552462|     99988.1|
|Cajero automatico|19692|1578.2967992078002|    99998.74|
|        App movil|78139|1523.8658099028673|    79999.99|
|    Domiciliacion|14874| 281.5033857738335|     50000.0|
+-----------------+-----+------------------+------------+



In [13]:
# Consulta 8: Clientes operando en MÚLTIPLES ciudades
spark.sql("""
    SELECT 
        id_cliente, 
        COUNT(DISTINCT ciudad) as numero_de_ciudades,
        COLLECT_SET(ciudad) as lista_ciudades_unicas
    FROM transacciones
    GROUP BY id_cliente
    HAVING numero_de_ciudades > 2
    ORDER BY numero_de_ciudades DESC
    LIMIT 10
""").show()

+----------+------------------+---------------------+
|id_cliente|numero_de_ciudades|lista_ciudades_unicas|
+----------+------------------+---------------------+
|      1084|                 6| [Leon, Guadalajar...|
|      1231|                 6| [Leon, Guadalajar...|
|      1092|                 6| [Leon, Guadalajar...|
|      1014|                 6| [Leon, Guadalajar...|
|      1105|                 6| [Leon, Guadalajar...|
|      1060|                 6| [Leon, Guadalajar...|
|      1132|                 6| [Leon, Guadalajar...|
|      1163|                 6| [Leon, Guadalajar...|
|      1174|                 6| [Leon, Guadalajar...|
|      1011|                 6| [Leon, Guadalajar...|
+----------+------------------+---------------------+

